<a href="https://colab.research.google.com/github/eli-osherovich/mnist-demo/blob/main/mnist_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist

In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-34f8f053-7af6-8ff2-fb80-f2005a952331)


In [3]:
tf.config.optimizer.set_jit(True)

In [4]:
# the data, split between train and test sets
dataset = fashion_mnist
dataset = mnist


(x_train, y_train), (x_test, y_test) = dataset.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [5]:
def gen_model(activation="relu"):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(28, 28, 1)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dropout(0.1))

  model.add(tf.keras.layers.Dense(128, activation=activation))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Dense(128, activation=activation))

  model.add(tf.keras.layers.Dense(128, activation=activation))
  model.add(tf.keras.layers.Dense(10))

  return model


def compile_model(model, lr=0.001):
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])
  

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=20,
      restore_best_weights=True,
)

In [6]:
%%time
model = gen_model()
compile_model(model)
model.fit(x_train, y_train,
          batch_size=128,
          epochs=1000,
          verbose=1,
          callbacks=[early_stop],
          validation_data=(x_test, y_test),
          validation_batch_size=2048)

Epoch 1/1000
469/469 [==============================] - 2s 5ms/step - loss: 0.4495 - accuracy: 0.8606 - val_loss: 0.1551 - val_accuracy: 0.9513
Epoch 2/1000
469/469 [==============================] - 3s 6ms/step - loss: 0.2238 - accuracy: 0.9313 - val_loss: 0.1324 - val_accuracy: 0.9598
Epoch 3/1000
469/469 [==============================] - 2s 3ms/step - loss: 0.1818 - accuracy: 0.9444 - val_loss: 0.1061 - val_accuracy: 0.9667
Epoch 4/1000
469/469 [==============================] - 1s 3ms/step - loss: 0.1598 - accuracy: 0.9504 - val_loss: 0.0910 - val_accuracy: 0.9727
Epoch 5/1000
469/469 [==============================] - 1s 3ms/step - loss: 0.1472 - accuracy: 0.9536 - val_loss: 0.0893 - val_accuracy: 0.9733
Epoch 6/1000
469/469 [==============================] - 1s 3ms/step - loss: 0.1334 - accuracy: 0.9582 - val_loss: 0.0852 - val_accuracy: 0.9734
Epoch 7/1000
469/469 [==============================] - 1s 3ms/step - loss: 0.1279 - accuracy: 0.9597 - val_loss: 0.0816 - val_accuracy:

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.05711185932159424
Test accuracy: 0.9844000339508057
